# Query caselaw-explorer API
---
Before loading the dependencies, please make sure to paste your user API key in the *.env* file under *COGNITO_USER_KEY*.
You can find the key in the bottom left menu of the app under *... > Preferences > Developer > API key*.
### Load dependencies & set up client

In [1]:
import boto3
import requests
from dotenv import load_dotenv
from os import getenv
from datetime import datetime
from os.path import join
from notebooks.api.gql_queries import *

load_dotenv()

refresh_token = getenv('COGNITO_USER_KEY')
client_id = getenv('COGNITO_CLIENT_ID')
endpoint = getenv('APPSYNC_ENDPOINT')

client = boto3.client('cognito-idp')
session = requests.Session()

### Authenticate (If token expired: re-run cell)

In [2]:
auth = client.initiate_auth(
    ClientId=client_id,
    AuthFlow='REFRESH_TOKEN',
    AuthParameters={'REFRESH_TOKEN': refresh_token}
)

### Function definitions

In [3]:
def execute(qgl_query):
    resp = session.request(
        url=endpoint,
        method='POST',
        headers={'authorization': auth['AuthenticationResult']['AccessToken']},
        json={'query': qgl_query}
    ).json()['data']
    query_name = list(resp.keys())[0]
    print(f'Query {query_name} executed.')
    return resp[query_name], query_name

def format_network(resp):
    nw = resp.copy()
    nw['nodes'] = [(lambda x: {'id': x['id'], 'data': json.loads(x['data'])})(x) for x in nw['nodes']]
    if nw['message'] != '':
        print('Query limit reached! Only partial result returned.\n')
    print('Network formatted.')
    return nw

def get_network_statistics(nodes, edges):
    nodes, b_name = execute(gql_batch_fetch_node_data(nodes, AttributesList.NETWORKSTATS))
    print('Required node metadata fetched.')
    query = gql_compute_networkstatistics(nodes, edges)
    stats, name = execute(query)
    print('Statistics calculated.')
    return json.loads(stats), name

def export_json(resp, file_name=''):
    path = join('output', f'{file_name}_{datetime.today().strftime("%Y-%m-%d_%H:%M:%S")}.json')
    with open(path, 'w') as f:
        json.dump(resp, f)
    print(f'Saved {file_name} to {path}.')

### Query network
At least one of the optional parameters must be selected.
Other parameters, if omitted, will assume their default value (see function definition).

For descriptions of the parameter options see *definitions > terminology > attribute_values.py*.

In [4]:
n_query = gql_query_network(
    attributes_to_fetch=AttributesList.ID,
    data_sources=[DataSource.RS],               #           list of DataSources (options so far: RS (Rechtspraak))
    eclis='ECLI:NL:HR:2004:AP0186',            # optional; string of eclis separated by space
    #keywords='gevaarzetting',                  # optional; string in Simple Query String Syntax (https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-simple-query-string-query.html)
    #articles='Artikel 3',                      # optional; string in Simple Query String Syntax, see user guide
    date_start='1900-01-01',                    #           string of format yyyy-mm-dd
    date_end='2021-12-31',                      #           string of format yyyy-mm-dd
    #instances=[Instance.HOGE_RAAD],            # optional; list of Instance options
    #domains=[Domain.STRAFRECHT],                # optional; list of Domain options
    doc_types=[DocType.DEC, DocType.OPI],                    #           list of DocTypes (options: DEC (decision), OPI (opinion))
    degrees_sources=1,                          #         response  int: degree of source citations
    degrees_targets=1                           #           int: degree of target citations
)

n_response, n_query_name = execute(n_query)

network = format_network(n_response)

export_json(network, n_query_name)

#print(json.dumps(network, indent=4))


Query queryNetworkByUserInput executed.
Network formatted.
Saved queryNetworkByUserInput to output/queryNetworkByUserInput_2022-01-05_12:10:55.json.


### Query network statistics

In [5]:
statistics, s_query_name = get_network_statistics(
    nodes=n_response['nodes'],
    edges=n_response['edges']
)

export_json(statistics, s_query_name)

Query batchFetchNodeData executed.
Required node metadata fetched.
Query computeNetworkStatistics executed.
Statistics calculated.
Saved computeNetworkStatistics to output/computeNetworkStatistics_2022-01-05_12:11:04.json.
